In [1]:
import torch
from transformers import MT5ForConditionalGeneration, MT5Config, MT5EncoderModel, MT5Tokenizer, Trainer, TrainingArguments
from progeny_tokenizer import TAPETokenizer
import numpy as np
import math
import random
import scipy
import time
import pandas as pd
from torch.utils.data import DataLoader, RandomSampler, Dataset, BatchSampler
import typing
from pathlib import Path
import argparse
from collections import OrderedDict
import pickle
import pathlib

import matplotlib.pyplot as plt

In [2]:
noperturb_results_tsv = 'generated_seqs/mcmc_SST5/SST5_mcmc_trainlabel2initseqs_20iter_temp01/20iter_temp01-mcmc_seqs.tsv'

In [3]:
perturb_results_tsvs = [
    'generated_seqs/mcmc_SST5/SST5_mcmc_trainlabel2initseqs_20iter_temp01/20iter_temp01-mcmc_seqs.tsv',
    'generated_seqs/mcmc_SST5/SST5_mcmc_trainlabel2initseqs_20iter_temp001/20iter_temp001-mcmc_seqs.tsv',
    'generated_seqs/mcmc_SST5/SST5_mcmc_trainlabel2initseqs_100iter_temp01/100iter_temp01-mcmc_seqs.tsv',
    'generated_seqs/mcmc_SST5/SST5_mcmc_trainlabel2initseqs_100iter_temp001/100iter_temp001-mcmc_seqs.tsv',
#     'generated_seqs/mcmc_SST5/SST5_mcmc_trainlabel2initseqs_200iter_temp01/200iter_temp01-mcmc_seqs.tsv',
#     'generated_seqs/mcmc_SST5/SST5_mcmc_trainlabel2initseqs_200iter_temp001/200iter_temp001-mcmc_seqs.tsv',
#     'generated_seqs/mcmc_SST5/SST5_mcmc_trainlabel2initseqs_500iter_temp01/500iter_temp01-mcmc_seqs.tsv',
#     'generated_seqs/mcmc_SST5/SST5_mcmc_trainlabel2initseqs_500iter_temp001/500iter_temp001-mcmc_seqs.tsv',
#     'generated_seqs/mcmc_SST5/SST5_mcmc_trainlabel2initseqs_1Kiter_temp01/1Kiter_temp01-mcmc_seqs.tsv',
#     'generated_seqs/mcmc_SST5/SST5_mcmc_trainlabel2initseqs_1Kiter_temp001/1Kiter_temp001-mcmc_seqs.tsv',
   
]

In [4]:
noperturb_df = pd.read_table(noperturb_results_tsv)

In [5]:
noperturb_df = noperturb_df.sort_values(by='disc_pred', ascending=False)
noperturb_df

,disc_pred,gt_class_pred,MT_edit_dist_vs_WT,generated_seq_ppl,gt_highest_prob,gt_2class_pred,gt_neg_prob,gt_pos_prob,generated_seq,accepted
0,14.137024,4,3,312.299530,0.637455,1,0.009894,0.754922,"Like the series, the movie is funny, susține, ...",0
1,13.965712,3,4,1075.685425,0.999706,1,0.000038,0.999732,"It's an unusual,gé bioRevelationologistsrama w...",0
2,13.942880,3,5,1371.973145,0.998177,1,0.000079,0.998265,"It's an unusual,Deutsch bioRevelationologistsr...",0
3,13.800753,3,4,763.473877,0.997488,1,0.000084,0.997577,"It's an unusual,Deutsch bioRevelationologistsr...",1
4,13.673590,2,5,490.791870,0.999582,0,0.000400,0.000018,"While reactions respective readers comedy, sc...",1
...,...,...,...,...,...,...,...,...,...,...
25979,-13.178345,2,7,4812.250000,0.999094,0,0.000875,0.000031,What disgust get... is Caddy EdenLigue payday ...,0
25980,-13.205957,2,7,8026.266113,0.999796,0,0.000180,0.000024,What disgust get... is Caddy EdenLigue payday ...,0
25981,-13.228817,1,2,2756.216553,0.998263,0,0.998304,0.000032,That is rack - damnbac!,1
25982,-13.321692,2,8,860.622192,0.999586,0,0.000391,0.000024,Since the trips hirings cântness is ironchy Ca...,0


In [6]:
print("np.max(noperturb_df['generated_seq_ppl']): ", np.max(noperturb_df['generated_seq_ppl']))
print("np.min(noperturb_df['generated_seq_ppl']): ", np.min(noperturb_df['generated_seq_ppl']))
print("np.mean(noperturb_df['generated_seq_ppl']): ", np.mean(noperturb_df['generated_seq_ppl']))


np.max(noperturb_df['generated_seq_ppl']):  16936044.0
np.min(noperturb_df['generated_seq_ppl']):  12.097006797790527
np.mean(noperturb_df['generated_seq_ppl']):  7280.434229748076


In [7]:
# noperturb_df['sent_delta'] = noperturb_df['gt_class_pred'] - noperturb_df['gen_input_seq_class']
# noperturb_df

# Compute % target class for noperturb

In [8]:
target_classes = [3,4]
topk_list = [10000, 1000, 100, 10]

percent_target_class = []

In [9]:
gt_class_preds = noperturb_df['gt_class_pred']
# gen_input_seq_classes = noperturb_df['gen_input_seq_class']
# sent_deltas = noperturb_df['sent_delta']

In [10]:
df = noperturb_df

# iterate through all perturbed result tsv files

In [11]:
for perturb_results_tsv in perturb_results_tsvs:
    print("*-"*30)
    print("perturb_results_tsv: ", perturb_results_tsv)
    
    perturb_df = pd.read_table(perturb_results_tsv)
    perturb_df = perturb_df.sort_values(by='disc_pred', ascending=False)
#     perturb_df['sent_delta'] = perturb_df['gt_class_pred'] - perturb_df['gen_input_seq_class']

    gt_class_preds = perturb_df['gt_class_pred']
#     gen_input_seq_classes = perturb_df['gen_input_seq_class']
#     sent_deltas = perturb_df['sent_delta']
    generated_seq_ppls = perturb_df['generated_seq_ppl']

    for target_class in target_classes:
        total_num  = len(perturb_df['gt_class_pred'])
        print("target_class: ", target_class)
        num_target_class = np.sum(perturb_df['gt_class_pred'] == target_class)

        percent_target_class = num_target_class / total_num *100
        print("percent_target_class: ", percent_target_class)

        for topk in topk_list:
            topk_gt_class_preds = gt_class_preds[:topk]
#             topk_sent_deltas = sent_deltas[:topk]


            topk_num  = len(topk_gt_class_preds)
            print("topk: ", topk)
    #         print("topk_gt_class_preds: ", topk_gt_class_preds)
            topk_num_target_class = np.sum(topk_gt_class_preds == target_class)
            topk_percent_target_class = topk_num_target_class / topk_num *100
#             print("topk_num_target_class: ", topk_num_target_class)
#             print("topk_num: ", topk_num)
            print("topk_percent_target_class: ", topk_percent_target_class)

#             topk_sent_delta_mean = np.mean(topk_sent_deltas)
    #         print("topk_sent_deltas: ", topk_sent_deltas)
#             print("topk_sent_delta_mean: ", topk_sent_delta_mean)

            print("*")
        print("--------------")


    print("-------For all target classes-------")
    print("target_classes: ", target_classes)
    total_num  = len(perturb_df['gt_class_pred'])
    num_target_class = np.sum(perturb_df['gt_class_pred'].isin(target_classes))

    percent_target_class = num_target_class / total_num *100
    print("percent_target_class: ", percent_target_class)

    for topk in topk_list:
        topk_gt_class_preds = gt_class_preds[:topk]
#         topk_sent_deltas = sent_deltas[:topk]
    
        topk_generated_seq_ppls = generated_seq_ppls[:topk]


        topk_num  = len(topk_gt_class_preds)
        print("topk: ", topk)
    #         print("topk_gt_class_preds: ", topk_gt_class_preds)
        topk_num_target_class = np.sum(topk_gt_class_preds.isin(target_classes))
        topk_percent_target_class = topk_num_target_class / topk_num *100
#         print("topk_num_target_class: ", topk_num_target_class)
#         print("topk_num: ", topk_num)
        print("topk_percent_target_class: ", topk_percent_target_class)

    
        topk_generated_seq_ppl_mean = np.mean(topk_generated_seq_ppls)
        topk_generated_seq_ppl_std = np.std(topk_generated_seq_ppls)
        print("topk_generated_seq_ppl_mean: ", topk_generated_seq_ppl_mean)
        print("topk_generated_seq_ppl_std: ", topk_generated_seq_ppl_std)
        
#         topk_sent_delta_mean = np.mean(topk_sent_deltas)
    #         print("topk_sent_deltas: ", topk_sent_deltas)
#         print("topk_sent_delta_mean: ", topk_sent_delta_mean)

        print("*")
    
    
    
    
    
        
    # E[% positive, strong-positive] computation

    df = perturb_df

    num_rounds = 100 # N
    round_pool_size = 1000
    topk = 100 # K
    main_pool_size = 25000
    target_classes = [3, 4]

    round_topk = {}
#     cols_to_sort = ['latent_head_pred']
    cols_to_sort = ['disc_pred']
    df_main_pool = df.sample(n=main_pool_size)


    print("--------------")
    print("E[% positive, strong-positive] computation")
    # print("Sorted by ", cols_to_sort)
    for col_to_sort in cols_to_sort:
        print("col_to_sort: ", col_to_sort)
        round_topk[col_to_sort] = {}
        for round_ind in range(num_rounds):
            sampled_rows = df_main_pool.sample(n=round_pool_size)
            sorted_sampled_rows = sampled_rows.sort_values(by=col_to_sort, ascending=False)[:topk]
            topk_rows = sorted_sampled_rows[:topk]
            round_topk[col_to_sort][round_ind] = {}



            for target_class in target_classes:
                total_num  = len(topk_rows['gt_class_pred'])
    #             print("target_class: ", target_class)
                num_target_class = np.sum(topk_rows['gt_class_pred'] == target_class)

                percent_target_class = num_target_class / total_num *100
    #             print("percent_target_class: ", percent_target_class)

                round_topk[col_to_sort][round_ind][target_class] = percent_target_class



    #         print("target_classes: ", target_classes)
            total_num  = len(topk_rows['gt_class_pred'])
            num_target_class = np.sum(topk_rows['gt_class_pred'].isin(target_classes))

            percent_target_class = num_target_class / total_num *100
    #         print("percent_target_class: ", percent_target_class)
            round_topk[col_to_sort][round_ind]['all'] = percent_target_class


        for target_class in target_classes:
            percent_values = []
            for round_ind in range(num_rounds):
                percent_values.append(round_topk[col_to_sort][round_ind][target_class])
            print("target_class: ", target_class)
            mean_percent_values = np.mean(percent_values)
            std_percent_values = np.std(percent_values)
            print("mean_percent_values: ", mean_percent_values)
            print("std_percent_values: ", std_percent_values)



        percent_values = []
        for round_ind in range(num_rounds):
            percent_values.append(round_topk[col_to_sort][round_ind]['all'])
        print("target_classes: ", target_classes)
        mean_percent_values = np.mean(percent_values)
        std_percent_values = np.std(percent_values)
        print("mean_percent_values: ", mean_percent_values)
        print("std_percent_values: ", std_percent_values)

*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
perturb_results_tsv:  generated_seqs/mcmc_SST5/SST5_mcmc_trainlabel2initseqs_20iter_temp01/20iter_temp01-mcmc_seqs.tsv
target_class:  3
percent_target_class:  3.6099137931034484
topk:  10000
topk_percent_target_class:  6.8500000000000005
*
topk:  1000
topk_percent_target_class:  16.7
*
topk:  100
topk_percent_target_class:  31.0
*
topk:  10
topk_percent_target_class:  40.0
*
--------------
target_class:  4
percent_target_class:  0.03848522167487685
topk:  10000
topk_percent_target_class:  0.1
*
topk:  1000
topk_percent_target_class:  0.8999999999999999
*
topk:  100
topk_percent_target_class:  4.0
*
topk:  10
topk_percent_target_class:  20.0
*
--------------
-------For all target classes-------
target_classes:  [3, 4]
percent_target_class:  3.648399014778325
topk:  10000
topk_percent_target_class:  6.950000000000001
topk_generated_seq_ppl_mean:  10493.211100110722
topk_generated_seq_ppl_std:  215055.98422064225
*
topk:  1000
t